# Transfrom All-Transactions House Price Index by County
### Data source: [GEOFRED](https://geofred.stlouisfed.org/map/?th=ylgn&cc=5&rc=false&im=fractile&sb&lng=-100.239&lat=41.558&zm=5&sl&sv&sti=942&rt=county&at=Not%20Seasonally%20Adjusted,%20Annual,%20Index%202000%3D100,%20no_period_desc&fq=Annual&dt=2020-01-01&am=Average&un=lin)

In [3]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import mitosheet
import os
import json
import re
from sqlalchemy import create_engine
import psycopg2
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
from mitosheet import *; register_analysis('UUID-24a5edab-3a03-48ab-aa0d-1b31c2acbcf5')

# Imported Resources\GeoFRED_All-Transactions_House_Price_Index_by_County_Index.xlsx into a DataFrame
sheet_df_dictonary = pd.read_excel('Resources\GeoFRED_All-Transactions_House_Price_Index_by_County_Index.xlsx', sheet_name=['house_price_index'], skiprows=0)
house_price_df = sheet_df_dictonary['house_price_index']
house_price_df.head()

,Series ID,Region Name,State code,State name,State and County,Region Code,1975,1976,1977,1978,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ATNHPIUS45001A,Abbeville County,SC,South Carolina,"Abbeville County, South Carolina",45001,NaN,NaN,NaN,NaN,...,128.12,123.05,123.16,130.71,131.96,133.75,142.15,152.61,152.99,165.44
1,ATNHPIUS22001A,Acadia Parish,LA,Louisiana,"Acadia Parish, Louisiana",22001,NaN,NaN,NaN,NaN,...,142.77,144.42,149.83,153.58,154.92,157.71,160.82,162.84,168.98,170.25
2,ATNHPIUS51001A,Accomack County,VA,Virginia,"Accomack County, Virginia",51001,NaN,NaN,NaN,NaN,...,167.34,163.35,162.22,160.17,163.32,163.24,168.20,165.83,171.98,182.96
3,ATNHPIUS16001A,Ada County,ID,Idaho,"Ada County, Idaho",16001,29.13,32.29,35.71,42.32,...,110.20,115.45,129.05,142.79,152.52,165.94,184.58,209.58,233.07,254.81
4,ATNHPIUS29001A,Adair County,MO,Missouri,"Adair County, Missouri",29001,NaN,NaN,NaN,NaN,...,113.79,116.24,116.75,118.19,117.48,119.61,121.36,127.29,128.14,128.20


### Process house_price_df data: remove unnecessary columns, change DataFrame name, add column for price change between 2015-19 

In [5]:
from mitosheet import *; register_analysis('UUID-4a9f5254-ece7-49cd-9498-e8705d0fab54')

# Deleted column State code from house_price_index
house_price_df.drop(['State code'], axis=1, inplace=True)

# Deleted column Series ID from house_price_index
house_price_df.drop(['Series ID'], axis=1, inplace=True)

# Deleted columns 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999 from house_price_index
house_price_df.drop(['1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999'], axis=1, inplace=True)

# Added column new-column-fup3 to house_price_df
house_price_df.insert(25, 'Price change % 2015-19', 0)

# Set new-column-fup3 in house_price_df to =2019/2014-1
house_price_df['Price change % 2015-19'] = house_price_df['2019']/house_price_df['2014']-1


In [6]:
house_price_df.head()

,Region Name,State name,State and County,Region Code,2000,2001,2002,2003,2004,2005,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Price change % 2015-19
0,Abbeville County,South Carolina,"Abbeville County, South Carolina",45001,100,107.98,111.81,114.34,120.85,124.86,...,123.05,123.16,130.71,131.96,133.75,142.15,152.61,152.99,165.44,0.170454
1,Acadia Parish,Louisiana,"Acadia Parish, Louisiana",22001,100,104.43,108.49,110.81,115.76,121.31,...,144.42,149.83,153.58,154.92,157.71,160.82,162.84,168.98,170.25,0.100273
2,Accomack County,Virginia,"Accomack County, Virginia",51001,100,110.40,117.26,128.84,150.11,182.78,...,163.35,162.22,160.17,163.32,163.24,168.20,165.83,171.98,182.96,0.073734
3,Ada County,Idaho,"Ada County, Idaho",16001,100,105.04,108.78,111.46,117.70,136.45,...,115.45,129.05,142.79,152.52,165.94,184.58,209.58,233.07,254.81,0.632257
4,Adair County,Missouri,"Adair County, Missouri",29001,100,102.37,102.96,105.43,107.36,113.61,...,116.24,116.75,118.19,117.48,119.61,121.36,127.29,128.14,128.20,0.084186


In [7]:
# Export house_price_df to a CSV file
house_price_df.to_csv('Resources\house_price_df.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'Resources\\house_price_df.csv'

### Sorting house_price_df to find counties that saw highest price increase during 2015-19 and charting them by state

In [8]:
from mitosheet import *; register_analysis('UUID-ea2ebf27-b6b1-4618-bbe1-e72cd571e0c2')

# Sorted Price change % 2015-19 in house_price_df in descending order
house_price_df_sorted = house_price_df.sort_values(by='Price change % 2015-19', ascending=False, na_position='last')

# Filtered Price change % 2015-19 in house_price_df
house_price_df_sorted = house_price_df_sorted[house_price_df_sorted['Price change % 2015-19'] > 0.4]

# Pivoted house_price_df into df2
unused_columns = house_price_df_sorted.columns.difference(set(['State name']).union(set([])).union(set({'Region Name'})))
tmp_df = house_price_df_sorted.drop(unused_columns, axis=1)
pivot_table = tmp_df.pivot_table(
    index=['State name'],
    values=['Region Name'],
    aggfunc={'Region Name': ['count']}
)
pivot_table.columns = [flatten_column_header(col) for col in pivot_table.columns.values]
house_price_df_pivot = pivot_table.reset_index()

# Sorted Region Name count in house_price_df_pivot in descending order
house_price_df_pivot = house_price_df_pivot.sort_values(by='Region Name count', ascending=False, na_position='last')


In [9]:
# Import plotly and create a figure
import plotly.graph_objects as go
fig = go.Figure()

# Add the bar chart traces to the graph. 
# Plotly documentationf for bar chart with direct labels https://plotly.com/python/bar-charts/#bar-chart-with-direct-labels
for column_header in ['State name']:
    fig.add_trace(
        go.Bar( 
            x=house_price_df_pivot[column_header],
            y=house_price_df_pivot['Region Name count'],
            name=str(column_header),
            text=house_price_df_pivot['Region Name count'],
            textposition='outside',
        )
    )

# Update the title and stacking mode of the graph
# See Plotly documentation for customizations: https://plotly.com/python/reference/bar/
fig.update_layout(
    xaxis_title="State name",
    yaxis_title="Number of counties",
    title="State with counties that saw >40% price increase over 2015-19",
    barmode='group',)
fig.show(renderer="iframe")

In [ ]:
# Render the Mitosheet with the data
# mitosheet.sheet(house_price_df)